# Gradient Boosted Trees

In [1]:
from fastbook import *
from utils import *
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
#from pactools.grid_search import GridSearchCVProgressBar
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
import sklearn

from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 200
pd.options.display.max_columns = 120

/opt/anaconda3/envs/fastaiV2/lib/python3.8/site-packages/fastbook/__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to.pkl').load()

In [3]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

## HistGradientClassifier

### Default parameters

In [4]:
def m_roc(y, xs):
    preds = clf.predict_proba(xs)
    preds_isFraud = preds[:,1]
    return roc_auc_score(y, preds_isFraud)

In [6]:
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=0)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

(0.9802477931308692, 0.855824270578817)

### Hyperparameter tuning

In [11]:
parameters1 = {
 'max_iter': [100,1000,1200,1500],
 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
 'max_depth': [None, 25, 50, 75],
 'l2_regularization': [0, 1.5],
 'max_bins': [255],
 'max_leaf_nodes': [20, 31, 40, 50, 60],
 'min_samples_leaf': [1, 5, 10, 20],
 'random_state': [33],
 }

In [7]:
parameters2 = {
 'max_iter': [100,1000],
 'learning_rate': [0.01, 0.1, 0.2, 0.3],
 'max_depth': [None, 25, 50],
 'l2_regularization': [0, 1.5],
 'max_bins': [255],
 'max_leaf_nodes': [20, 31, 40],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [23]:
parameters3 = {
 'max_iter': [1000, 2000],
 'learning_rate': [0.005, 0.01, 0.1],
 'max_depth': [15, 25, 35],
 'l2_regularization': [1.5, 2.0],
 'max_bins': [255],
 'max_leaf_nodes': [40, 45, 50],
 'min_samples_leaf': [5, 7, 10],
 'random_state': [33],
 }

#### GridSearchCV

In [110]:
clf = GridSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy'), parameters, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [111]:
# took over 40 mins, than stopped it.
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

UsageError: Line magic function `%%time` not found.


In [8]:
clf = GridSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy'), parameters2, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [12]:
RandomForestClassifier?

In [9]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

KeyboardInterrupt: 

In [91]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

#### RandomizedSearchCV

In [16]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy'), parameters2, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [17]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 7min 44s, sys: 2min 23s, total: 10min 8s
Wall time: 11min 5s


(HistGradientBoostingClassifier(l2_regularization=1.5, learning_rate=0.01,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=1000, max_leaf_nodes=40,
                                min_samples_leaf=5, random_state=33),
 0.9004300572018135,
 {'random_state': 33,
  'min_samples_leaf': 5,
  'max_leaf_nodes': 40,
  'max_iter': 1000,
  'max_depth': 25,
  'max_bins': 255,
  'learning_rate': 0.01,
  'l2_regularization': 1.5})

In [21]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=0, l2_regularization=1.5,
                             learning_rate=0.01, max_depth=25, max_iter=1000, max_leaf_nodes=40,
                             min_samples_leaf=5, random_state=33)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

CPU times: user 6min 1s, sys: 2min 5s, total: 8min 7s
Wall time: 1min 8s


(0.9755758828468353, 0.8634802312949047)

In [22]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=0, l2_regularization=1.5,
                             learning_rate=0.01, max_depth=25, max_iter=1000, max_leaf_nodes=40,
                             min_samples_leaf=5, random_state=None)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

CPU times: user 6min 16s, sys: 2min 5s, total: 8min 22s
Wall time: 1min 11s


(0.9784349524797529, 0.8643483751952322)

In [24]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy'), parameters3, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [25]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 5min 17s, sys: 1min 50s, total: 7min 8s
Wall time: 28min 50s


(HistGradientBoostingClassifier(l2_regularization=2.0, learning_rate=0.01,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=2000, max_leaf_nodes=40,
                                min_samples_leaf=7, random_state=33),
 0.900838109847753,
 {'random_state': 33,
  'min_samples_leaf': 7,
  'max_leaf_nodes': 40,
  'max_iter': 2000,
  'max_depth': 25,
  'max_bins': 255,
  'learning_rate': 0.01,
  'l2_regularization': 2.0})

In [36]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=0, l2_regularization=2.0,
                             learning_rate=0.01, max_depth=25, max_iter=2000, max_leaf_nodes=40,
                             min_samples_leaf=7, random_state=None)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

CPU times: user 6min 38s, sys: 2min 9s, total: 8min 47s
Wall time: 1min 14s


(0.9799498632581533, 0.8659922390606523)

In [31]:
parameters4 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.02],
 'max_depth': [25, 30],
 'l2_regularization': [2.0, 2.5, 3.0],
 'max_bins': [255],
 'max_leaf_nodes': [37, 40, 43],
 'min_samples_leaf': [7, 8, 9],
 'random_state': [33],
 }

In [32]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy'), parameters4, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [33]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 2min, sys: 43.5 s, total: 2min 43s
Wall time: 22min 15s


(HistGradientBoostingClassifier(l2_regularization=2.5, learning_rate=0.02,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=1000, max_leaf_nodes=43,
                                min_samples_leaf=8, random_state=33),
 0.9019179542023302,
 {'random_state': 33,
  'min_samples_leaf': 8,
  'max_leaf_nodes': 43,
  'max_iter': 1000,
  'max_depth': 25,
  'max_bins': 255,
  'learning_rate': 0.02,
  'l2_regularization': 2.5})

In [37]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=0, l2_regularization=2.5,
                             learning_rate=0.02, max_depth=25, max_iter=2000, max_leaf_nodes=40,
                             min_samples_leaf=8, random_state=None)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

CPU times: user 2min 45s, sys: 53.1 s, total: 3min 38s
Wall time: 31 s


(0.9717789053240151, 0.8614802948697504)

In [41]:
parameters5 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.02, 0.03],
 'max_depth': [25, 27],
 'l2_regularization': [2.4, 2.5, 2.6],
 'max_bins': [255],
 'max_leaf_nodes': [43, 44, 45],
 'random_state': [33],
 }

In [42]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy'), parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [40]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 4min 32s, sys: 1min 37s, total: 6min 10s
Wall time: 23min 48s


(HistGradientBoostingClassifier(l2_regularization=2.0, learning_rate=0.02,
                                loss='binary_crossentropy', max_depth=30,
                                max_iter=1000, max_leaf_nodes=43,
                                min_samples_leaf=8, random_state=33),
 0.9014128006684489,
 {'random_state': 33,
  'min_samples_leaf': 8,
  'max_leaf_nodes': 43,
  'max_iter': 1000,
  'max_depth': 30,
  'max_bins': 255,
  'learning_rate': 0.02,
  'l2_regularization': 2.0})

In [ ]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=0, l2_regularization=2.0,
                             learning_rate=0.02, max_depth=30, max_iter=2000, max_leaf_nodes=43,
                             min_samples_leaf=8, random_state=None)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)